In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset

In [3]:
import torch
from modeling_llama import LlamaForCausalLM
from transformers import AutoTokenizer

/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


(Re-)Loading modeling...


/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
torch.nn.Linear.reset_parameters = lambda x: None
model = LlamaForCausalLM.from_pretrained('NousResearch/Llama-2-13b-chat-hf', torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
model = model.to('cuda:0').eval()

In [6]:
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-13b-chat-hf')

In [7]:
prompts = []

In [8]:
xsum = load_dataset('xsum').shuffle(4242)
cnn = load_dataset('cnn_dailymail', '3.0.0').shuffle(4242)

/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/datasets/load.py:1429: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [9]:
for i in range(4):
    item = cnn['train'][i+100]
    cnn_context = 'Article: ' + item['article'] + '\nSummary: ' + item['highlights'].replace('\n', '')
    
    item = cnn['train'][i]
    prompt = cnn_context + '\nArticle: ' + item['article'] + '\nSummary:'
    prompts.append(prompt)

In [10]:
for i in range(4):
    item = xsum['train'][i+100]
    xsum_context = 'Article: ' + item['document'] + '\nSummary: ' + item['summary'].replace('\n', '')
    
    item = xsum['train'][i]
    prompt = xsum_context + '\nArticle: ' + item['document'] + '\nSummary:'
    prompts.append(prompt)

In [11]:
from searching import LayerSkippingSearching

In [12]:
layer_searching = LayerSkippingSearching(model, tokenizer, prompts, evaluate_config={"generate_fn": "essg", "max_new_tokens": 32})

In [13]:
layer_searching.probe([8,10,15,18,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,], [])
layer_searching.probe([3, 5, 6, 8, 10, 11, 14, 15, 18, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37], [6, 9, 10, 11, 15, 24, 25, 27, 28, 35])

In [15]:
layer_searching.search(2)

|   iter    |  target   |    x0     |    x1     |    x10    |    x11    |    x12    |    x13    |    x14    |    x15    |    x16    |    x17    |    x18    |    x19    |    x2     |    x20    |    x21    |    x22    |    x23    |    x24    |    x25    |    x26    |    x27    |    x28    |    x29    |    x3     |    x30    |    x31    |    x32    |    x33    |    x34    |    x35    |    x36    |    x37    |    x38    |    x39    |    x4     |    x40    |    x41    |    x42    |    x43    |    x44    |    x45    |    x46    |    x47    |    x48    |    x49    |    x5     |    x50    |    x51    |    x52    |    x53    |    x54    |    x55    |    x56    |    x57    |    x58    |    x59    |    x6     |    x60    |    x61    |    x62    |    x63    |    x64    |    x65    |    x66    |    x67    |    x68    |    x69    |    x7     |    x70    |    x71    |    x72    |    x73    |    x74    |    x75    |    x76    |    x77    |    x78    |    x79    |    x8     |    x9     |
--------------

([3,
  5,
  6,
  8,
  10,
  11,
  14,
  15,
  18,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  33,
  34,
  35,
  36,
  37],
 [6, 9, 10, 11, 15, 24, 25, 27, 28, 35])

In [16]:
layer_searching.get_solution()

([3,
  5,
  6,
  8,
  10,
  11,
  14,
  15,
  18,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  33,
  34,
  35,
  36,
  37],
 [6, 9, 10, 11, 15, 24, 25, 27, 28, 35])